In [1]:
from enum import auto, Enum
import os
import tempfile
import math

import numpy as np
from tqdm.auto import tqdm
import rasterio
from rasterio.windows import Window, union
import torch
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from pathlib import Path
from hanning import get_bartlett_hann_kernel, Kernel, BartlettHannKernel

In [2]:
CLIP_SIZE = 1024
NUM_CLASSES = 2
DEVICE = torch.device('cuda')


# IMG_PATH = "/mnt/aco-uvic/Taylor_temp/23_3031_01_NW_Calvert_CCKelp_ORTHO_CSRS_UTM9_CGG2013a_subset3.tif"
# OUTPUT_PATH = "/mnt/aco-uvic/Taylor_temp/23_3031_01_NW_Calvert_CCKelp_ORTHO_CSRS_UTM9_CGG2013a_subset3_kelp_pa.tif"

# IMG_PATH = "/mnt/aco-uvic/Taylor_temp/23_3033_01_Triquet_CCKelp_ORTHO_CSRS_UTM9_CGG2013a_subset.tif"
# OUTPUT_PATH = "/mnt/aco-uvic/Taylor_temp/23_3033_01_Triquet_CCKelp_ORTHO_CSRS_UTM9_CGG2013a_subset_kelp_pa.tif"

# IMG_PATH = "/mnt/aco-uvic/2023_Acquisitions/02_Processed/23_3033_01_Triquet_CCKelp/02_ORTHO/01_Products/23_3033_01_Triquet_CCKelp_ORTHO_CSRS_UTM9_CGG2013a_FINAL.tif"
# OUTPUT_PATH = "/mnt/data/Taylor/23_3033_01_Triquet_CCKelp_ORTHO_CSRS_UTM9_CGG2013a_kelp_pa.tif"

IMG_PATH = "/mnt/aco-uvic/2023_Acquisitions/02_Processed/23_3058_01_SalishSea/02_ORTHO/01_Products/23_3058_01_SalishSea_ORTHO_CSRS_UTM10_CGG2013a_with_sfm_DEM_cog.tif"
OUTPUT_PATH = "/mnt/data/Taylor/23_3058_01_SalishSea_ORTHO_CSRS_UTM10_CGG2013a_with_sfm_DEM_kelp_pa.tif"

MODEL_WEIGHTS = "./UNET_Resnet34_kelp_presence_aco_jit_miou=0.8323.pt"

# Classification v1.0

In [32]:
# with open(MODEL_WEIGHTS, "rb") as f: 
#     model = torch.jit.load(f, map_location=DEVICE).eval()

In [33]:
# # For each set of 4x4 image windows (less/more?), forward prop through net to get prediction. Stride is half of window size (e.g. 2x2)
# def generate_windows(tile_size, img_height, img_width):
#     stride = tile_size // 2
    
#     for row_start in range(0, img_height, stride):
#         for col_start in range(0, img_width, stride):
#             row_stop = min(row_start + tile_size, img_height)
#             col_stop = min(col_start + tile_size, img_width)
#             yield Window.from_slices((row_start, row_stop), (col_start, col_stop))

In [34]:
# from rasterio.io import MemoryFile

# ZEROS_CHIP = np.zeros((NUM_CLASSES, CLIP_SIZE//2, CLIP_SIZE//2), dtype=np.uint8)


# # with MemoryFile() as output_temp:
# with tempfile.NamedTemporaryFile(suffix=".tif", delete=False) as output_temp_f:
#     output_temp = output_temp_f.name
#     print(output_temp)
    
#     # Write a new single band geotiff with the same size and CRS as the src image and all pixels==0
#     with rasterio.open(IMG_PATH) as src:
#         # Initialize temp register file
#         profile = src.profile
#         profile.update({ "count": NUM_CLASSES, "height": int(CLIP_SIZE*1.5), "dtype": "float32", "bigtiff": "IF_SAFER" })
#         with rasterio.open(output_temp, "w", **profile) as dest:
#             pass

#         # Initialize final output file
#         profile = src.profile
#         profile.update({ "count": 1, "dtype": "uint8", "bigtiff": "IF_SAFER" })
#         with rasterio.open(OUTPUT_PATH, 'w', **profile) as dest:
#             pass

    
#     with rasterio.open(IMG_PATH) as src:
#         windows = list(generate_windows(CLIP_SIZE, src.profile['height'], src.profile['width']))
#         last_col_off = windows[-1].col_off
#         last_row_off = windows[-1].row_off
        
#         for window in tqdm(windows):
#             x = torch.tensor(src.read(window=window), device=DEVICE, dtype=torch.uint8)
            
#             # zero pad to size
#             _, th, tw = x.shape
#             dh = CLIP_SIZE - th
#             dw = CLIP_SIZE - tw
#             x = torch.nn.functional.pad(x, (0,dw,0,dh), mode='constant', value=0)
            
#             if (x == 0).all():    
#                 # Mock softmax outputs for BG with 100% confidence
#                 pred = torch.zeros((NUM_CLASSES, CLIP_SIZE, CLIP_SIZE), device=DEVICE)
#                 pred[0] = 1
#             else:
#                 # Preprocessing
#                 # to float
#                 x = x.to(torch.float) / 255
#                 # min-max scale
#                 min_, _ = torch.kthvalue(x.flatten().unique(), 2)
#                 max_ = x.flatten().max()
#                 x = torch.clamp((x - min_) / (max_ - min_ + 1e-8), 0, 1)
    
#                 # batch_size=1
#                 x = x.unsqueeze(dim=0)
                
#                 # Prediction
#                 with torch.no_grad():
#                     pred = model.forward(x).squeeze(dim=0)
            
#             # Convolve with hann window
#             kernel = get_bartlett_hann_kernel(
#                 size=CLIP_SIZE,
#                 device=DEVICE,
#                 is_top=(window.row_off == 0),
#                 is_bottom=(window.row_off == last_row_off),
#                 is_left=(window.col_off == 0),
#                 is_right=(window.col_off == last_col_off),
#             )
#             kernel_pred = torch.mul(kernel, pred)
                
#             # Strip padding
#             kernel_pred = kernel_pred[:, :th, :tw]
            
#             # Read any existing data writen to the output and add to the new data
#             with rasterio.open(output_temp, 'r+') as register, rasterio.open(OUTPUT_PATH, 'r+', **profile) as dest:
#                 # 1. Read and write to the registry
#                 # | | | |
#                 # |x|x| |
#                 # |x|x| |
#                 win = Window(col_off=window.col_off, row_off=CLIP_SIZE//2, height=window.height, width=window.width)
#                 with torch.no_grad():
#                     existing_values = torch.tensor(register.read(window=win), device=DEVICE)
#                     logits_xxxx = existing_values + kernel_pred
#                 # Write the new data to the output
#                 register.write(logits_xxxx.detach().cpu().numpy(), window=win)

#                 # 2. Write the values that are information complete (The upper left of the above window)
#                 # | | | |
#                 # |x| | |
#                 # | | | |
#                 logits_x = logits_xxxx[:,:min(CLIP_SIZE//2, window.height),:min(CLIP_SIZE//2, window.width)]
#                 win = Window(col_off=window.col_off, row_off=window.row_off, height=min(CLIP_SIZE//2, window.height), width=min(CLIP_SIZE//2, window.width))
#                 dest.write(logits_x.argmax(axis=0).detach().cpu().numpy(), indexes=1, window=win)

#                 # 3. Update registry
#                 # | | | |      |x| | |
#                 # |x|o| |  ->  |x|o| |
#                 # |x|o| |      | |o| |
#                 logits_xx = logits_xxxx[:,:,:min(CLIP_SIZE//2, window.width)]
#                 write_win = Window(col_off=window.col_off, row_off=0, height=window.height, width=min(CLIP_SIZE//2, window.width))
#                 zero_win = Window(col_off=window.col_off, row_off=CLIP_SIZE, height=CLIP_SIZE//2, width=min(CLIP_SIZE//2, window.width))
#                 register.write(logits_xx.detach().cpu().numpy(), window=write_win)
#                 register.write(ZEROS_CHIP, window=zero_win)
                
#                 # If last col, do the same as above for the right half of the window
#                 # I think this never happens..? The last tile will always be < CLIP_SIZE//2 wide. Except maybe if it is exactly CLIP_SIZE//2?
#                 if window.col_off == last_col_off and window.width > CLIP_SIZE//2:
#                     import pdb; pdb.set_trace()
#                     # 2. Write the values that are information complete (The upper left of the above window)
#                     # | | | |
#                     # | | |x|
#                     # | | |o|  # This section saved for later, to prevent extra read
#                     logits_x = logits_xxxx[:,:min(CLIP_SIZE//2, window.height),min(CLIP_SIZE//2, window.width):]
#                     win = Window(col_off=(window.col_off+CLIP_SIZE//2), row_off=window.row_off, height=min(CLIP_SIZE//2, window.height), width=(window.width-CLIP_SIZE//2))
#                     dest.write(logits_x.argmax(axis=0).detach().cpu().numpy(), indexes=1, window=win)

#                     # 3. Update registry
#                     # | |o| |      | |o|x|
#                     # | |o|x|  ->  | |o|x|
#                     # | | |x|      | | | |
#                     logits_xx = logits_xxxx[:,:,min(CLIP_SIZE//2, window.width):]
#                     write_win = Window(col_off=(window.col_off+CLIP_SIZE//2), row_off=0, height=window.height, width=(window.width-CLIP_SIZE//2))
#                     zero_win = Window(col_off=(window.col_off+CLIP_SIZE//2), row_off=CLIP_SIZE, height=CLIP_SIZE//2, width=(window.width-CLIP_SIZE//2))
#                     register.write(logits_xx.detach().cpu().numpy(), window=write_win)
#                     register.write(ZEROS_CHIP, window=zero_win)

#             # Postprocess logits
#             # with :
#             #     windows = list(src.block_windows(1))
#             #     for ji, window in tqdm(windows):
#             #         logits = np.stack([src.read(k+1, window=window) for k in range(NUM_CLASSES)], axis=0)
#             #         preds = logits.argmax(axis=0)
#             #         dest.write(preds, indexes=1, window=window)

/tmp/tmppm7gmsgl.tif


  0%|          | 0/9 [00:00<?, ?it/s]

# Classification v1.1

In [3]:
class MemoryRegister(object):
    def __init__(self, image_path: str, num_classes: int, window_size: int, device: torch.device.type):
        super().__init__()
        self.image_path = image_path
        self.n = num_classes
        self.ws = window_size
        self.hws = window_size//2
        self.device = device

        # Copy metadata from img
        with rasterio.open(image_path, 'r') as src:
            self.width = math.ceil(src.width / self.ws) * self.ws
        self.height = self.ws
        self.register = np.zeros((self.n, self.height, self.width+self.hws))

    def __enter__(self):
        return self

    def __exit__(self, *args, **kwargs):
        del self.register
    
    @property
    def _zero_chip(self):
        return torch.zeros((self.n, self.hws, self.hws), dtype=torch.float, device=self.device)
        
    def step(self, new_logits: torch.Tensor, img_window: Window):
        # 1. Read data from the registry to update with the new logits
        # |a|b| |
        # |c|d| |
        with torch.no_grad():
            logits_abcd = torch.tensor(self.register[:,:,img_window.col_off:img_window.col_off+self.ws], device=self.device)
            logits_abcd += new_logits

        # Move data around and write to the registry to make new space for the next row of processing windows      
        # |c|b| | + pop a
        # |0|d| |
        logits_a = logits_abcd[:,:self.hws,:self.hws]
        logits_c = logits_abcd[:,self.hws:,:self.hws]
        logits_c0 = torch.concat([logits_c, self._zero_chip], dim=1).detach().cpu().numpy()
        logits_bd = logits_abcd[:,:,self.hws:].detach().cpu().numpy()

        # write c0
        self.register[:,:,img_window.col_off:img_window.col_off+self.hws] = logits_c0

        # write bd
        col_off_bd = img_window.col_off+self.hws
        win_bd = Window(col_off=col_off_bd, row_off=0, height=self.ws, width=self.hws)
        self.register[:,:,col_off_bd:col_off_bd+self.hws] = logits_bd
        
        # Return the information complete predictions (argmax of a, stripped over overflowing padding)
        preds_win = Window(col_off=img_window.col_off, row_off=img_window.row_off, 
                           height=min(self.hws, img_window.height), width=min(self.hws, img_window.width))
        preds = logits_a[:, :img_window.height, :img_window.width].argmax(axis=0)

        return preds, preds_win

In [4]:
class RasterRegister(object):
    def __init__(self, image_path: str, num_classes: int, window_size: int, device: torch.device.type):
        super().__init__()
        self.image_path = image_path
        self.tf = tempfile.NamedTemporaryFile(suffix=".tif", delete=False)
        self.n = num_classes
        self.ws = window_size
        self.hws = window_size//2
        self.device = device

        # Copy metadata from img
        with rasterio.open(self.image_path) as src:
            working_profile = src.profile
            self.height = self.ws
            self.width = math.ceil(src.width / self.ws) * self.ws
            working_profile.update({
                "count": self.n, "height": self.height, "width": self.width, 
                "dtype": "float32", "bigtiff": "IF_SAFER" 
            })
            
            # Initialize empty registry file
            with rasterio.open(self.tf.name, "w", **working_profile) as dest:
                pass

        # Open registry file in append mode 
        self.register = rasterio.open(self.tf.name, "r+")

    def __enter__(self):
        return self

    def __exit__(self, *args, **kwargs):
        self.register.close()
        self.tf.close()
        os.unlink(self.tf.name)
    
    @property
    def _zero_chip(self):
        return torch.zeros((self.n, self.hws, self.hws), dtype=torch.float, device=self.device)
        
    def step(self, new_logits: torch.Tensor, img_window: Window):
        # 1. Read data from the registry to update with the new logits
        # |a|b| |
        # |c|d| |
        win = Window(col_off=img_window.col_off, row_off=0, height=self.ws, width=self.ws)
        with torch.no_grad():
            logits_abcd = torch.tensor(self.register.read(window=win, boundless=True, fill_value=0), device=self.device)
            logits_abcd += new_logits

        # Move data around and write to the registry to make new space for the next row of processing windows      
        # |c|b| | + pop a
        # |0|d| |
        logits_a = logits_abcd[:,:self.hws,:self.hws]
        logits_c = logits_abcd[:,self.hws:,:self.hws]
        logits_c0 = torch.concat([logits_c, self._zero_chip], dim=1).detach().cpu().numpy()
        logits_bd = logits_abcd[:,:,self.hws:].detach().cpu().numpy()

        win_c0 = Window(col_off=img_window.col_off, row_off=0, height=self.ws, width=self.hws)
        self.register.write(logits_c0, window=win_c0)

        col_off_bd = img_window.col_off+self.hws
        if col_off_bd + self.hws <= self.width:
            win_bd = Window(col_off=col_off_bd, row_off=0, height=self.ws, width=self.hws)
            self.register.write(logits_bd, window=win_bd)
        
        # Return the information complete predictions (argmax of a, stripped over overflowing padding)
        preds_win = Window(col_off=img_window.col_off, row_off=img_window.row_off, 
                           height=min(self.hws, img_window.height), width=min(self.hws, img_window.width))
        preds = logits_a[:, :img_window.height, :img_window.width].argmax(axis=0)

        return preds, preds_win
    

In [5]:
class HannWindowSegmentation(object):
    def __init__(self, model: torch.nn.Module, kernel: Kernel,
                 num_classes: int, window_size: int, device: torch.device.type):
        super().__init__()
        self.model = model
        self.kernel = kernel(window_size, device)
        self.n = num_classes
        self.ws = window_size
        self.hws = window_size//2
        self.device = device

    @property
    def _pure_black_pred(self):
        """Shortcut prediction for any tiles that are all black. Equal to a pred of BG for all pixels"""
        pred = torch.zeros((self.n, self.ws, self.ws), device=self.device)
        pred[0] = 1
        return pred

    def _window_generator(self, img_height: int, img_width: int):
        for row_start in range(0, img_height, self.hws):
            for col_start in range(0, img_width, self.hws):
                row_stop = min(row_start + self.ws, img_height)
                col_stop = min(col_start + self.ws, img_width)
                yield Window.from_slices((row_start, row_stop), (col_start, col_stop))

    def _pad_tile(self, x: torch.Tensor) -> torch.Tensor:
        # zero pad image data to consistent window size
        _, th, tw = x.shape
        return torch.nn.functional.pad(x, (0, self.ws - tw, 0, self.ws - th), mode='constant', value=0)

    def _preprocess(self, x: torch.Tensor) -> torch.Tensor:
        # to float
        x = x.to(torch.float) / 255
        # min-max scale
        min_, _ = torch.kthvalue(x.flatten().unique(), 2)
        max_ = x.flatten().max()
        x = torch.clamp((x - min_) / (max_ - min_ + 1e-8), 0, 1)

        # batch_size=1
        x = x.unsqueeze(dim=0)
        return x
    
    def __call__(self, img_path: str, output_path: str):
        # Initialize the output file and working file
        self._init_output_file(img_path, output_path)

        # Classification
        with rasterio.open(img_path) as src, MemoryRegister(img_path, self.n, self.ws, self.device) as register, rasterio.open(output_path, 'r+') as dest:
            windows = list(self._window_generator(src.height, src.width))
            last_col_off = windows[-1].col_off
            last_row_off = windows[-1].row_off
            
            for window in tqdm(windows):
                x = torch.tensor(src.read(window=window), device=self.device, dtype=torch.uint8)
                x = self._pad_tile(x)
                
                if (x == 0).all():    
                    # Mock softmax outputs for BG with 100% confidence
                    pred = self._pure_black_pred
                else:
                    # Prediction
                    x = self._preprocess(x)
                    with torch.no_grad():
                        pred = model.forward(x).squeeze(dim=0)
                
                # Multiply by hann window
                with torch.no_grad():
                    kernel_pred = self.kernel(
                        pred, 
                        top=(window.row_off == 0), 
                        bottom=(window.row_off == last_row_off),
                        left=(window.col_off == 0),
                        right=(window.col_off == last_col_off)
                    )
                
                # Update the registry with intermediate data and recieve complete predictions
                preds, pred_win = register.step(kernel_pred, window)
                dest.write(preds.detach().cpu().numpy(), indexes=1, window=pred_win)
        
    
    def _init_output_file(self, img_path: str, output_path: str):
        """Write a new single band geotiff with the same size and CRS as the src image and all pixels==0"""
        with rasterio.open(img_path) as src:     
            # Initialize final output file
            output_profile = src.profile
            output_profile.update({ "count": 1, "dtype": "uint8", "bigtiff": "IF_SAFER" })
            with rasterio.open(output_path, 'w', **output_profile) as dest:
                pass
                
    @classmethod
    def run(cls, img_path: str, output_path: str, model: torch.nn.Module, kernel: Kernel, num_classes: int, window_size: int, device: torch.device.type):
        self = cls(model, kernel, num_classes, window_size, device)
        return self(img_path, output_path)

with open(MODEL_WEIGHTS, "rb") as f: 
    model = torch.jit.load(f, map_location=DEVICE).eval()
    
HannWindowSegmentation.run(IMG_PATH, OUTPUT_PATH, model, BartlettHannKernel, NUM_CLASSES, CLIP_SIZE, DEVICE)

  0%|          | 0/73584 [00:00<?, ?it/s]

In [7]:
!jupyter nbconvert --to script Inference.ipynb

[NbConvertApp] Converting notebook Inference.ipynb to script
[NbConvertApp] Writing 19588 bytes to Inference.py


In [3]:
# Add Gradio interface